In [ ]:
from imutils import paths
import os
import librosa
import numpy as np


def getMFCC(i):
  signal, sr = librosa.load(i, sr=16000)

  # MFCC를 통한 특징 벡터 추출 (n_mfcc=36)
  MFCCs = librosa.feature.mfcc(signal, sr, n_fft=400, hop_length=160, n_mfcc=36)
  return MFCCs

forder = '/content/drive/MyDrive/cut2'
files = paths.list_files(forder)

dataset=[]
label=[] 

for i in files:
  mfcc=getMFCC(i)
  dataset.append(mfcc.reshape(-1)) # 1차원 배열로 반환
  label.append((os.path.basename(i)).split('_')[1]) # 파일의 이름에서 감정을 나타내는 부분을 추출해서 저장


from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
encoder = LabelEncoder()

encoder.fit(label)
y = encoder.transform(label)


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 훈련 데이터 split
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size = 0.2, random_state = 42, shuffle = True)

# 커널 설정
clf = SVC(kernel = 'rbf', probability=True)

# MinMaxScaler를 통해 특징 벡터 scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 조정된 데이터로 SVM 학습
clf.fit(X_train_scaled, y_train)

# 스케일 조정된 테스트 세트의 정확도
print(clf.score(X_test_scaled, y_test))

